In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [10]:
import tensorflow as tf
import tensorflow_hub as hub

from sklearn.model_selection import train_test_split

In [9]:
import json
from google.colab import files
uploaded = files.upload()

Saving Sarcasm_Headlines_Dataset_v2.csv to Sarcasm_Headlines_Dataset_v2.csv


In [12]:
import io
df = pd.read_csv(io.BytesIO(uploaded['Sarcasm_Headlines_Dataset_v2.csv']))

In [13]:
df.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In [14]:
x = df['is_sarcastic']
y = df['headline']

In [15]:
trainX, testX, trainY, testY = train_test_split(x,y, test_size=0.2)

In [16]:
model_tokenizer = Tokenizer()
model_tokenizer.fit_on_texts(trainY)

trainingSequences = model_tokenizer.texts_to_sequences(trainY)
trainingPadded = pad_sequences(trainingSequences, 100)

testingSequences = model_tokenizer.texts_to_sequences(testY)
testingPadded = pad_sequences(testingSequences, 100)

In [17]:
model = Sequential()

model.add(Embedding(100000, 12, input_length=100))
model.add(LSTM(128))
model.add(Dense(27, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
model.summary()

trainingHistory = model.fit(trainingPadded, trainX, epochs=5, validation_data=(testingPadded, testX), batch_size=300)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 12)           1200000   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               72192     
_________________________________________________________________
dense (Dense)                (None, 27)                3483      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 28        
Total params: 1,275,703
Trainable params: 1,275,703
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
77/77 [==============================] - 50s 611ms/step - loss: 0.6653 - accuracy: 0.5867 - val_loss: 0.4397 - val_accuracy: 0.8125
Epoch 2/5
77/77 [==============================] - 48s 618ms/step - loss: 0.3704 - accuracy: 0.8597 - 